# Visualize the effect of network behavioral properties on distributed algorithms


In [1]:
# for interactive plots
%matplotlib notebook

from pydistsim.network import NetworkGenerator
from pydistsim.demo_algorithms.broadcast import Flood
from pydistsim.simulation import Simulation
from pydistsim.network.behavior import (
    NetworkBehaviorModel,
    random_loss,
)

from pydistsim.gui import drawing as draw
from IPython.display import HTML

## Introduction

First, we will define the network and the distributed algorithm that we will use to illustrate the effect of network
properties on distributed algorithms. We will use the **Flood** as the distributed algorithm and a 4 by 4 grid as
the network.


In [2]:
net = NetworkGenerator.generate_grid_network(16)
sim = Simulation(net, (Flood,))

Now, let's create a helper function to visualize the network and the algorithm, with the behavioral properties as a
parameter. We also define whether or not the visualization will display the internal clock of the nodes.


In [3]:
def make_vid(properties: NetworkBehaviorModel, clock_as_label: bool = False) -> HTML:
    # Set the network behavior properties
    net.behavioral_properties = properties

    # Dont interrupt the simulation when a restriction is violated (Total Reliability in this case)
    sim.check_restrictions = False
    sim.reset()

    if clock_as_label:
        # Show the clock of each node as a label, for that, we need to pass a lambda function to the node_labels parameter
        kwargs = {"show_labels": True, "node_labels": lambda: {node: node.clock for node in net.nodes}}
    else:
        kwargs = {"show_labels": False}

    # Create the animation
    anim = draw.create_animation(sim, **kwargs)
    video = anim.to_html5_video()

    return HTML(video)

## Ideal Communication

First, the simplest scenario is when the network has no loss and no delay. In this case, the algorithm will appear to be
executing in a synchronous manner.


In [4]:
make_vid(NetworkBehaviorModel.IdealCommunication)

## Delay

Now, let's visualize the effect of delay on the distributed algorithm. We will introduce a random delay in every edge of
the network. The delay will be between 1 and 16 time units.


In [5]:
# RandomDelayCommunication introduces random delays in the communication, proportional to the size of the network
make_vid(NetworkBehaviorModel.RandomDelayCommunication)

## Loss

Here we will visualize the effect of loss on the distributed algorithm. We will introduce a random loss in every edge of
the network. The loss will happen with a probability of 0.1.


In [7]:
# UnlikelyRandomLossCommunication sets the probability of a message being lost to 0.1
make_vid(NetworkBehaviorModel.UnlikelyRandomLossCommunication)

We now increase the loss probability to 0.5. It should be low enough to allow the algorithm to terminate on most
executions.


In [18]:
# Create our own NetworkBehaviorModel, with a message loss probability of 0.5
make_vid(NetworkBehaviorModel(message_loss_indicator=random_loss(0.5)))

Finally, we increase the loss probability to 0.7. The algorithm will surely not terminate.


In [16]:
# Now with much higher message loss probability
make_vid(NetworkBehaviorModel(message_loss_indicator=random_loss(0.7)))

## Clock skew


Finally, we will visualize the effect of clock skew on the distributed algorithm. We will start to use the parameter
`clock_as_label` to display the internal clock of the nodes in the visualization.


The `clock_increment` parameter of `NetworkBehaviorModel` will be used to simulate the clock skew. It will be added
to the internal clock of the nodes in every simulation step. For a fully synchronous network, the clock skew should be a
constant value for all nodes.

The next cell does exactly that. We set an increment of 1.


In [9]:
make_vid(
    NetworkBehaviorModel(clock_increment=lambda node: 1),
    clock_as_label=True,
)

The increment now is set to be a random value between 1 and 2. This will make the nodes' internal clocks to drift apart
slowly.


In [10]:
import random


make_vid(
    NetworkBehaviorModel(clock_increment=lambda node: random.randint(1, 2)),
    clock_as_label=True,
)

To exaggerate the effect of clock skew, we set the increment to be a random value between 1 and 5.


In [11]:
make_vid(
    NetworkBehaviorModel(clock_increment=lambda node: random.randint(1, 5)),
    clock_as_label=True,
)

# Everything together

Just for fun, let's visualize the effect of all the disruptive network properties together.


In [39]:
make_vid(
    NetworkBehaviorModel(
        message_loss_indicator=random_loss(0.3),  # 30% packet loss
        message_delay_indicator=lambda network, message: random.randint(1, 5),  # 1 to 5 time units delay
        clock_increment=lambda node: random.randint(1, 3),  # 1 to 3 time units per clock tick
    ),
    clock_as_label=True,
)